In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 1000
    xa1 = 2*torch.rand(Nb, dimension) - 1.0
    xa1[:, 0] = -torch.ones(Nb)
    
    xa2 = 2*torch.rand(Nb, dimension) - 1.0
    xa2[:, 1:3] = xa1[:, 1:3]
    xa2[:, 0] = torch.ones(Nb)
    
    xb1 = 2*torch.rand(Nb, dimension) - 1.0
    xb1[:, 1] = -torch.ones(Nb)
    
    xb2 = 2*torch.rand(Nb, dimension) - 1.0
    xb2[:, 0] = xb1[:, 0] 
    xb2[:, 2:3] = xb1[:, 2:3]
    xb2[:, 1] = torch.ones(Nb)
    
    xc1 = 2*torch.rand(Nb, dimension) - 1.0
    xc1[:, 2] = -torch.ones(Nb)
    
    xc2 = 2*torch.rand(Nb, dimension) - 1.0
    xc2[:, 0:1] = xc1[:, 0:1]
    xc2[:, 3] = xc1[:, 3]
    xc2[:, 2] = torch.ones(Nb)
    
    xd1 = 2*torch.rand(Nb, dimension) - 1.0
    xd1[:, 3] = -torch.ones(Nb)
    
    xd2 = 2*torch.rand(Nb, dimension) - 1.0
    xd2[:, 0:2] = xd1[:, 0:2]  
    xd2[:, 3] = torch.ones(Nb)
    xa1 = xa1.cuda()
    xa2 = xa2.cuda()
    xb1 = xb1.cuda()
    xb2 = xb2.cuda()
    xc1 = xc1.cuda()
    xc2 = xc2.cuda()
    xd1 = xd1.cuda()
    xd2 = xd2.cuda()
    
    pxa = ((model(xa1) - model(xa2))**2).cuda() 
    pxb = ((model(xb1) - model(xb2))**2).cuda()  
    pxc = ((model(xc1) - model(xc2))**2).cuda()  
    pxd = ((model(xd1) - model(xd2))**2).cuda()  
    part_2 = torch.sum((pxa + pxb + pxc + pxd)[:,0])/2/xa1.size()[0]/4
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx4[:, 3] = torch.ones(xb1.size()[0])
    
    dxa = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 - (model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2)**2).cuda()  
    dxb = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 - (model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2)**2).cuda()  
    dxc = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 - (model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2)**2).cuda()  
    dxd = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 - (model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2)**2).cuda()  
    part_3 = torch.sum((dxa + dxb + dxc + dxd)[:,0])/2/xb1.size()[0]/4
    
    lambda1 = 1.0
    lambda2 = 0.5
    return part_1 + lambda1 * part_2 + lambda2 * part_3

In [10]:
Data_size = 2000
def Gendata():
    x = 2*torch.rand(Data_size, dimension) - 1.0
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Periodic.npy", error_save)
np.save("DRM_relative_error_4D_Periodic.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(80.1365, device='cuda:0')
current relative error is:  tensor(2.1096, device='cuda:0')
current epoch is:  50
current loss is:  tensor(2.0221, device='cuda:0')
current relative error is:  tensor(1.0010, device='cuda:0')
current epoch is:  100
current loss is:  tensor(-2.1832, device='cuda:0')
current relative error is:  tensor(0.8835, device='cuda:0')
current epoch is:  150
current loss is:  tensor(-3.7567, device='cuda:0')
current relative error is:  tensor(0.8541, device='cuda:0')
current epoch is:  200
current loss is:  tensor(-4.0617, device='cuda:0')
current relative error is:  tensor(0.8110, device='cuda:0')
current epoch is:  250
current loss is:  tensor(-5.7454, device='cuda:0')
current relative error is:  tensor(0.7887, device='cuda:0')
current epoch is:  300
current loss is:  tensor(-6.4067, device='cuda:0')
current relative error is:  tensor(0.7558, device='cuda:0')
current epoch is:  350
current loss is:  tensor(-7.4920, device='c

current epoch is:  3050
current loss is:  tensor(-19.0007, device='cuda:0')
current relative error is:  tensor(0.1176, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(-16.7430, device='cuda:0')
current relative error is:  tensor(0.1091, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(-18.5028, device='cuda:0')
current relative error is:  tensor(0.1087, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(-18.6040, device='cuda:0')
current relative error is:  tensor(0.1178, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(-18.5733, device='cuda:0')
current relative error is:  tensor(0.1109, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(-19.0923, device='cuda:0')
current relative error is:  tensor(0.1060, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(-17.0346, device='cuda:0')
current relative error is:  tensor(0.1020, device='cuda:0')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-19.0036, device='cuda:0')
current relative error is:  tensor(0.0912, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(-19.0410, device='cuda:0')
current relative error is:  tensor(0.0836, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(-19.6987, device='cuda:0')
current relative error is:  tensor(0.0776, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(-17.7040, device='cuda:0')
current relative error is:  tensor(0.0771, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(-18.6862, device='cuda:0')
current relative error is:  tensor(0.0750, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(-17.0036, device='cuda:0')
current relative error is:  tensor(0.0785, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(-20.6591, device='cuda:0')
current relative error is:  tensor(0.0890, device='cuda:0')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-19.9856, device='cuda:0')
current relative error is:  tensor(0.0668, device='cuda:0')
current epoch is:  9200
current loss is:  tensor(-20.2303, device='cuda:0')
current relative error is:  tensor(0.0627, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(-19.1205, device='cuda:0')
current relative error is:  tensor(0.0731, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(-20.3100, device='cuda:0')
current relative error is:  tensor(0.0664, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(-18.8928, device='cuda:0')
current relative error is:  tensor(0.0703, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(-20.8732, device='cuda:0')
current relative error is:  tensor(0.0685, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(-19.8946, device='cuda:0')
current relative error is:  tensor(0.0719, device='cuda:0')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-19.3976, device='cuda:0')
current relative error is:  tensor(0.0628, device='cuda:0')
current epoch is:  12200
current loss is:  tensor(-18.5130, device='cuda:0')
current relative error is:  tensor(0.0721, device='cuda:0')
current epoch is:  12250
current loss is:  tensor(-18.8952, device='cuda:0')
current relative error is:  tensor(0.0678, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(-19.9014, device='cuda:0')
current relative error is:  tensor(0.0507, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(-20.0449, device='cuda:0')
current relative error is:  tensor(0.0854, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(-19.9803, device='cuda:0')
current relative error is:  tensor(0.0621, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(-18.9078, device='cuda:0')
current relative error is:  tensor(0.0751, device='cuda:0')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-19.3554, device='cuda:0')
current relative error is:  tensor(0.0586, device='cuda:0')
current epoch is:  15200
current loss is:  tensor(-18.2879, device='cuda:0')
current relative error is:  tensor(0.0617, device='cuda:0')
current epoch is:  15250
current loss is:  tensor(-20.4881, device='cuda:0')
current relative error is:  tensor(0.0613, device='cuda:0')
current epoch is:  15300
current loss is:  tensor(-20.1204, device='cuda:0')
current relative error is:  tensor(0.0734, device='cuda:0')
current epoch is:  15350
current loss is:  tensor(-19.3384, device='cuda:0')
current relative error is:  tensor(0.0729, device='cuda:0')
current epoch is:  15400
current loss is:  tensor(-19.3419, device='cuda:0')
current relative error is:  tensor(0.0626, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(-19.2031, device='cuda:0')
current relative error is:  tensor(0.0622, device='cuda:0')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-18.6379, device='cuda:0')
current relative error is:  tensor(0.0478, device='cuda:0')
current epoch is:  18200
current loss is:  tensor(-19.4079, device='cuda:0')
current relative error is:  tensor(0.0641, device='cuda:0')
current epoch is:  18250
current loss is:  tensor(-19.5548, device='cuda:0')
current relative error is:  tensor(0.0675, device='cuda:0')
current epoch is:  18300
current loss is:  tensor(-19.2425, device='cuda:0')
current relative error is:  tensor(0.0800, device='cuda:0')
current epoch is:  18350
current loss is:  tensor(-17.7739, device='cuda:0')
current relative error is:  tensor(0.0938, device='cuda:0')
current epoch is:  18400
current loss is:  tensor(-18.6876, device='cuda:0')
current relative error is:  tensor(0.0590, device='cuda:0')
current epoch is:  18450
current loss is:  tensor(-20.7833, device='cuda:0')
current relative error is:  tensor(0.0543, device='cuda:0')
current epoch is:  18500
current loss is: